# Final Project Notebook — Aidan

_Date: August 17, 2025_


This notebook consolidates prior milestones into a complete end‑to‑end project:

- Import & setup
- Data loading
- Initial exploratory data analysis (EDA)
- Data cleaning
- Analysis & visualization to answer **at least four** business questions
- Conclusions, recommendations, and future work





## 1) Project Overview

**Client (who benefits):**  An online retail company looking to optimize its product mix, customer outreach, and
international expansion

**Business context / value:** Inventory optimization through identification of best-selling items and seasonal trends
* Targeted marketing via customer segmentation (RFM)
* Strategic growth by pinpointing lucrative overseas markets
* Pricing strategy insights by understanding price–volume dynamics>_
**Questions to answer:
1. _Q1_ Which products generate the most revenue?
2. _Q2_  How do sales trend over time?

3. _Q3_  How can customers be segmented by RFM (Recency, Frequency, Monetary)?
4. _Q4_ 4. Which countries (outside the UK) contribute most to international revenue?
5. ._Q5_  What’s the relationship between unit price and quantity sold?

**Tools**
- Python (pandas, numpy, matplotlib)
- Jupyter Notebook
- Optional: seaborn, scikit


## Load your data

Choose one of the two options below:

- **Option A (Local path):** set `DATA_PATH` to your file (CSV or XLSX).
- **Option B (Colab upload):** leave `DATA_PATH=None` and run; you'll be prompted to upload.


In [ ]:
import pandas as pd
import io, os

# Detect Colab
try:
    from google.colab import files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

DEFAULT_NAME = "online_retail_sample_10k_clean.csv"  # change if needed

def _read_any(path_or_buf, filename_hint=""):
    """Read CSV/CSV.GZ/Excel into a DataFrame."""
    name = (filename_hint or (str(path_or_buf) if isinstance(path_or_buf, str) else ""))
    lower = name.lower()
    if lower.endswith((".xlsx", ".xls")):
        return pd.read_excel(path_or_buf)
    comp = "gzip" if lower.endswith(".gz") else "infer"
    return pd.read_csv(path_or_buf, compression=comp, low_memory=False, encoding="latin1")

# ==== PROMPT & LOAD ====
if os.path.exists(DEFAULT_NAME):
    print(f"Found {DEFAULT_NAME} in the working directory. Loading it…")
    df_raw = _read_any(DEFAULT_NAME, filename_hint=DEFAULT_NAME)

elif IN_COLAB:
    print("Please upload your CSV / CSV.GZ / XLSX file…")
    uploaded = files.upload()
    if not uploaded:
        raise SystemExit("No file uploaded.")
    name, data = next(iter(uploaded.items()))
    buf = io.BytesIO(data)
    df_raw = _read_any(buf, filename_hint=name)
    print(f"Loaded: {name}  -> shape={df_raw.shape}")

else:
    # Local Jupyter: file dialog with fallback to manual path
    path = ""
    try:
        import tkinter as tk
        from tkinter import filedialog
        tk.Tk().withdraw()
        path = filedialog.askopenfilename(
            title="Select CSV/CSV.GZ or Excel file",
            filetypes=[("CSV", "*.csv"), ("Compressed CSV", "*.csv.gz"),
                       ("Excel", "*.xlsx *.xls"), ("All files", "*.*")]
        )
    except Exception:
        pass
    if not path:
        path = input("Enter path to your CSV/CSV.GZ or Excel file: ").strip()
    if not path:
        raise SystemExit("No file selected.")
    df_raw = _read_any(path, filename_hint=os.path.basename(path))
    print(f"Loaded: {os.path.basename(path)}  -> shape={df_raw.shape}")

print("Initial shape:", df_raw.shape)
df_raw.head(3)


Please upload your CSV / CSV.GZ / XLSX file…


#### Detected data‑loader snippet(s) from Milestone 4 (for reference)

## Initial EDA

In [ ]:
# Basic shape & columns
try:
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
except NameError:
    print("⚠️ Define df first in the Data Loading section.")

# Types, info, samples
try:
    print("\nDTypes:")
    print(df.dtypes)
    print("\nInfo:")
    print(df.info())
    display(df.head(10))
    display(df.tail(5))
    display(df.sample(min(5, len(df))))
except Exception as e:
    print("EDA preview error:", e)

# Descriptive statistics (numeric & object separately for clarity)
try:
    display(df.describe(include=[np.number]).T)
    display(df.describe(include=[object]).T)
except Exception as e:
    print("Describe error:", e)


## 5) Data Cleaning PT.1


Describe the cleaning decisions you made and **why** (NaNs, outliers, type fixes, merges, renames, splits, standardization):

- _Example_: Dropped rows with invalid dates because ...
- _Example_: Imputed missing unit price with median by country because ...
- _Example_: Standardized country names ...



### From dedicated cleaning notebook

In [ ]:
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

In [ ]:
# Example cleaning: Drop rows missing CustomerID (common in this dataset for incomplete transactions)
if 'CustomerID' in df.columns:
    df = df.dropna(subset=['CustomerID'])
print('After dropping missing CustomerID:', df.shape)

In [ ]:
# Fill missing descriptions if StockCode is present but Description is NaN (optional strategy)
if 'Description' in df.columns and 'StockCode' in df.columns:
    df['Description'] = df['Description'].fillna('No description')
df['Description'].isna().sum()

In [ ]:
dup_count = df.duplicated().sum()
print('Duplicate rows:', dup_count)
df = df.drop_duplicates()
print('After removing duplicates:', df.shape)

In [ ]:
# Check basic stats
df[['Quantity', 'UnitPrice']].describe()

In [ ]:
# Remove rows with non-positive Quantity or UnitPrice (common cleaning choice for sales analysis)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
print('After removing non-positive Quantity/UnitPrice:', df.shape)

In [ ]:
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
df.columns

In [ ]:
cleaned_path = 'online_retail_cleaned.csv'
df.to_csv(cleaned_path, index=False)
cleaned_path

Analysis

## Clean the data PT2.

- Remove credit notes (InvoiceNo starting with 'C') if present  
- Keep only positive quantities and prices  
- Drop rows missing the SKU/StockCode


In [ ]:
df = df_raw.copy()

def _find_col(df, options):
    for c in df.columns:
        if c.lower() in [o.lower() for o in options]:
            return c
    return None

sku_col = _find_col(df, ['StockCode', 'SKU', 'stockcode', 'sku'])
if sku_col is None:
    raise ValueError('Could not find a SKU column. Expected one of: StockCode, SKU')

invoice_col = _find_col(df, ['InvoiceNo', 'Invoice'])
qty_col     = _find_col(df, ['Quantity', 'Qty'])
price_col   = _find_col(df, ['UnitPrice', 'Price'])
if qty_col is None or price_col is None:
    raise ValueError("Could not find quantity/price columns. Need 'Quantity' & 'UnitPrice' (or 'Qty'/'Price').")

if invoice_col is not None:
    df[invoice_col] = df[invoice_col].astype(str)
    df = df[~df[invoice_col].str.startswith('C', na=False)]

df = df[(df[qty_col] > 0) & (df[price_col] > 0)]
df = df.dropna(subset=[sku_col]).copy()

df['Revenue'] = df[qty_col] * df[price_col]

print(df.shape)
df.head(3)


In [ ]:
# Imports
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For optional Colab upload
try:
    from google.colab import files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)


## Q1 —  Which products generate the most revenue?
This chart ranks the top 10 SKUs by total revenue, highlighting the products that drive sales and should be prioritized for inventory and promotions.

In [ ]:
topn = 10
revenue_by_sku = (
    df.groupby(sku_col, as_index=False)['Revenue']
      .sum()
      .sort_values('Revenue', ascending=False)
      .head(topn)
)

display(revenue_by_sku)

plt.figure(figsize=(10, 5))
plt.bar(revenue_by_sku[sku_col].astype(str), revenue_by_sku['Revenue'])
plt.title('Top-10 SKUs by Revenue — Aidan')
plt.xlabel('SKU')
plt.ylabel('Revenue (raw scale)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## Q2 — 2. How do sales trend over time?
This chart shows monthly revenue, revealing seasonal peaks and dips to guide staffing, inventory, and campaign timing.


In [ ]:
# Simple sales trends & seasonality (keep it basic)
import pandas as pd
import matplotlib.pyplot as plt
from calendar import month_abbr

# use df if it exists, else df_raw
try:
    data = df.copy()
except NameError:
    data = df_raw.copy()

# basic prep
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
data = data.dropna(subset=['InvoiceDate'])
data['Revenue'] = data['Quantity'] * data['UnitPrice']
data = data[data['Revenue'] > 0]

# monthly revenue (line)
monthly = data.set_index('InvoiceDate').resample('MS')['Revenue'].sum()
plt.plot(monthly.index, monthly.values)
plt.title('Monthly Revenue — Aidan')
plt.xlabel('Month'); plt.ylabel('Revenue')
plt.tight_layout(); plt.show()

# average revenue by month-of-year (bar)
moy = monthly.groupby(monthly.index.month).mean()
labels = [month_abbr[m] for m in range(1, 13)]
vals = [moy.get(m, 0) for m in range(1, 13)]
plt.bar(labels, vals)
plt.title('Average Revenue by Month — Aidan')
plt.xlabel('Month'); plt.ylabel('Avg Monthly Revenue')
plt.tight_layout(); plt.show()

print('Peak month (avg):', month_abbr[moy.idxmax()])



### Q3. _Which countries (outside the UK) contribute most to international revenue?e_

> This chart ranks non-UK countries by revenue to pinpoint the strongest international markets for localized marketing and distribution.



In [ ]:
# Top non-UK countries by total revenue
req = {"Country","Revenue"}
if not req.issubset(df.columns):
    raise ValueError(f"Missing columns: {req - set(df.columns)}")

intl = df[df["Country"].astype(str).str.lower() != "united kingdom"]
top_countries = (
    intl.groupby("Country")["Revenue"]
        .sum()
        .sort_values(ascending=False)
        .head(10)
        .reset_index()
)

print(top_countries)

fig, ax = plt.subplots()
ax.bar(top_countries["Country"].astype(str), top_countries["Revenue"])
ax.set_title("International Revenue (Top Countries, excl. UK) — Aidan")
ax.set_xlabel("Country")
ax.set_ylabel("Total Revenue")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


### Q4. _What’s the relationship between unit price and quantity sold?_
This scatterplot shows the relationship between unit price and quantity sold; a downward pattern suggests higher prices reduce volume, while an upward pattern suggests the opposite.


In [ ]:
# Scatter: UnitPrice vs Quantity (clip extreme outliers to see the pattern)
req = {"UnitPrice","Quantity"}
if not req.issubset(df.columns):
    raise ValueError(f"Missing columns: {req - set(df.columns)}")

tmp = df[(df["UnitPrice"] > 0) & (df["Quantity"] > 0)].copy()

# Clip extreme tails (99th percentile) for readability
q_price = tmp["UnitPrice"].quantile(0.99)
q_qty = tmp["Quantity"].quantile(0.99)
tmp = tmp[(tmp["UnitPrice"] <= q_price) & (tmp["Quantity"] <= q_qty)]

print("Points plotted:", len(tmp))

fig, ax = plt.subplots()
ax.scatter(tmp["UnitPrice"], tmp["Quantity"], alpha=0.3, s=10)
ax.set_title("Unit Price vs Quantity Sold — Aidan")
ax.set_xlabel("Unit Price")
ax.set_ylabel("Quantity")
plt.tight_layout()
plt.show()



### Q5.  How can customers be segmented by RFM (Recency, Frequency, Monetary?)
This chart groups customers by RFM (High-Value, At-Risk, Occasional), showing where to focus VIP rewards, win-back offers, and nurture campaigns.

_

In [ ]:
# Simple RFM segmentation (one cell)
import pandas as pd
import matplotlib.pyplot as plt

# 1) Prep
data = df.copy()
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"], errors="coerce")
data["Quantity"] = pd.to_numeric(data["Quantity"], errors="coerce")
data["UnitPrice"] = pd.to_numeric(data["UnitPrice"], errors="coerce")
data["Revenue"] = data["Quantity"] * data["UnitPrice"]
data = data.dropna(subset=["CustomerID","InvoiceDate"])
data = data[data["Revenue"] > 0]

# 2) Snapshot date and R/F/M
snapshot = data["InvoiceDate"].max() + pd.Timedelta(days=1)
g = data.groupby("CustomerID")
recency   = g["InvoiceDate"].max().apply(lambda d: (snapshot - d).days)
frequency = (g["InvoiceNo"].nunique() if "InvoiceNo" in data.columns else g.size())
monetary  = g["Revenue"].sum()

rfm = pd.DataFrame({"Recency": recency, "Frequency": frequency, "Monetary": monetary})

# 3) Scores (1–5). For Recency, lower days = higher score.
rfm["R_Score"] = pd.qcut(rfm["Recency"],   5, labels=[5,4,3,2,1]).astype(int)
# Use pd.cut for Frequency to handle potential duplicate bin edges
rfm["F_Score"] = pd.cut(rfm["Frequency"], 5, labels=[1,2,3,4,5], duplicates='drop').astype(int)
rfm["M_Score"] = pd.qcut(rfm["Monetary"],  5, labels=[1,2,3,4,5]).astype(int)
rfm["RFM_Sum"] = rfm[["R_Score","F_Score","M_Score"]].sum(axis=1)

# 4) Simple segments
def seg(row):
    if row["RFM_Sum"] >= 13: return "High-Value"
    if row["R_Score"] <= 2 and row["F_Score"] <= 2: return "At-Risk"
    return "Occasional"
rfm["Segment"] = rfm.apply(seg, axis=1)

# 5) Output + quick chart
counts = rfm["Segment"].value_counts()
print(counts)
counts.plot(kind="bar")
plt.title("Customer Segments (RFM) — Aidan")
plt.xlabel("Segment"); plt.ylabel("Customers")
plt.tight_layout(); plt.show()

rfm.head()
tmp = df[(df['UnitPrice']>0)&(df['Quantity']>0)].copy()
up99, q99 = tmp['UnitPrice'].quantile(0.99), tmp['Quantity'].quantile(0.99)
tmp = tmp[(tmp['UnitPrice']<=up99)&(tmp['Quantity']<=q99)]
r = tmp['UnitPrice'].corr(tmp['Quantity'])

import numpy as np
b1 = np.polyfit(np.log(tmp['UnitPrice']), np.log(tmp['Quantity']), 1)[0]  # ~elasticity
print(f"Price–Qty: r = {r:.2f}; elasticity ≈ {b1:.2f}")


### Interpretation of Graphs

Top-10 Products (bar): Revenue is concentrated in a small set of SKUs, highlighting clear priorities for inventory and promotion.

Monthly Revenue (line): Sales move in a seasonal pattern with distinct peaks and troughs across the timeline.

Monthly Revenue + 3-mo MA (line): The smoothed line reveals the underlying direction of sales beyond month-to-month noise.

Avg Revenue by Month-of-Year (bar): Certain calendar months consistently outperform others, confirming seasonality at the month level.

Avg Revenue by Day-of-Week (bar): Sales are uneven across the week, with some days reliably stronger and others consistently softer.

RFM Segments (bar): Most customers cluster into Occasional/At-Risk groups while a smaller High-Value segment delivers outsized impact.

Top Countries ex-UK (bar): A few non-UK markets dominate international revenue, indicating where localization will pay off first.

Unit Price vs Quantity (scatter): Higher prices generally align with lower quantities (and vice-versa), with clusters showing common price–volume patterns.



## Conclusions & Recommendations

**Insights discovered:**
Q1 (Top products): #1 21175 at 3,656; #2 85099B $3,056, #3 47566 $2,116 (also 22112 $1,514, 85123A $1,195, 22730 $1,126).

Q2 (Trend): Peak month (avg) = November.

Q3 (International): Top non-UK markets: EIRE $6,090, Netherlands $4,975, Germany $4,357, France $3,892.

Q4 From 9,630 transactions, unit price and quantity show a moderate negative relationship (r = −0.26); a log–log fit gives elasticity ≈ −0.54 (≈1% price ↑ → ~0.54% units ↓). Most sales cluster at <$5 and <10 units, while items >$10 rarely sell in high quantities.

Q5 (RFM): Occasional 59.8% (1,439), At-Risk 40.0% (961), High-Value 0.2% (5) of 2,405 customers

**Recommendations to client:**
- Q1: Prioritize inventory and promotions for the top SKUs; verify margins before scaling spend.
- Q2: Staff up and build inventory ahead of peak months; time campaigns 2–3 weeks before peaks.
- Q3: Offer VIP perks to High-Value customers, run win-back offers for At-Risk, and nurture Occasional buyers.
- Q4: Localize marketing, currency, and shipping for top non-UK countries; explore regional distributors.
- Q5: A/B test price tiers/bundles and avoid blanket discounts on price-insensitive items.

**Future work:**
- Q1: Analyze SKU-level profitability and attachment (bundle) opportunities.
- Q2: Build a seasonal forecast with holiday flags to predict next-quarter revenue.
- Q3: Test segmentation-specific campaigns and measure lift vs. control groups.
- Q4: Compare CAC, LTV, and margins by country; pilot localized landing pages.
- Q5: Estimate price elasticity by SKU controlling for seasonality and promotions.


## Appendix — Reproducibility

In [ ]:
import platform, sys, pandas, numpy, matplotlib
print("Python:", platform.python_version())
print("Platform:", platform.platform())
print("pandas:", pandas.__version__)
print("numpy:", numpy.__version__)
print("matplotlib:", matplotlib.__version__)


Citations
Dataset: Online Retail (UCI ML Repository)

APA: Chen, D. (2015). Online Retail [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5BW33.
UCI Machine Learning Repository

MLA: Chen, Daqing. Online Retail. UCI Machine Learning Repository, 2015. doi:10.24432/C5BW33.